# Colab 3: DPO Reinforcement Learning with SmolLM2-135M using Unsloth

This notebook demonstrates **DPO (Direct Preference Optimization)** - a reinforcement learning technique for aligning language models with human preferences.

### What is DPO?
- **Preference Learning**: Learns from pairs of responses (good vs bad)
- **Simpler than RLHF**: No reward model needed, direct optimization
- **Human Alignment**: Makes models prefer helpful, harmless, honest responses
- **Efficient**: Faster and more stable than traditional RL methods

### How DPO Works:
1. Start with a supervised fine-tuned (SFT) model
2. Show pairs of responses: one preferred, one rejected
3. Train model to increase probability of preferred responses
4. Decrease probability of rejected responses
5. Use KL divergence to prevent drift from original model

### Key Features:
- Model: `unsloth/SmolLM2-135M-Instruct` (pre-trained SFT model)
- Dataset: Ultrafeedback binarized preferences (500 pairs)
- Training time: ~2-3 minutes on free Colab T4 GPU
- Method: DPO with LoRA adapters

### What You'll Learn:
1. Difference between SFT and preference-based training
2. How to prepare preference datasets
3. Configuring DPO training
4. Understanding reward metrics
5. Evaluating preference alignment

## Step 1: Install Unsloth with RL Support

We need the reinforcement learning components.

In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install trl -U

## Step 2: Verify GPU and Setup

In [ ]:
import torch
from unsloth import FastLanguageModel, PatchDPOTrainer

# Patch DPO for Unsloth optimizations
PatchDPOTrainer()

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: Tesla T4
CUDA version: 12.6


## Step 3: Load Pre-trained SFT Model

### Important: DPO requires a supervised fine-tuned (SFT) model!
- We use `SmolLM2-135M-Instruct` which is already instruction-tuned
- DPO refines this model's behavior using preference data
- Think of it as: SFT teaches "what" to say, DPO teaches "how" to say it

In [ ]:
# Configuration
max_seq_length = 2048
dtype = None
load_in_4bit = True  # Use 4-bit quantization for efficiency

# Load SFT model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/SmolLM2-135M-Instruct",  # Pre-trained SFT model
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print(f"\n✅ SFT model loaded successfully!")
print(f"Model: {model.config._name_or_path}")

==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/423 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]


✅ SFT model loaded successfully!
Model: unsloth/SmolLM2-135M-Instruct


## Step 4: Apply LoRA Adapters for DPO

We'll use LoRA to make training efficient.

In [ ]:
# Configure LoRA for DPO
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  # LoRA rank
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

print("✅ LoRA adapters configured for DPO training!")

Unsloth 2025.11.2 patched 30 layers with 30 QKV layers, 30 O layers and 30 MLP layers.


✅ LoRA adapters configured for DPO training!


## Step 5: Load Preference Dataset

### DPO Dataset Format:
Each example has 3 fields:
- **prompt**: The input question/instruction
- **chosen**: The preferred response (higher quality)
- **rejected**: The rejected response (lower quality)

### Example:
```
Prompt: "How do I learn Python?"
Chosen: "Start with basics like variables and loops. Practice with projects..."
Rejected: "Just Google it."
```

In [ ]:
from datasets import load_dataset

# Load preference dataset (first 500 pairs for quick training)
dataset = load_dataset(
    "argilla/ultrafeedback-binarized-preferences-cleaned",
    split="train[:500]"
)

print(f"Dataset loaded: {len(dataset)} preference pairs")
print(f"\nDataset columns: {dataset.column_names}")

# Show first example structure
print(f"\nFirst example structure:")
print(f"Keys: {list(dataset[0].keys())}")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/143M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60917 [00:00<?, ? examples/s]

Dataset loaded: 500 preference pairs

Dataset columns: ['source', 'prompt', 'chosen', 'chosen-rating', 'chosen-model', 'rejected', 'rejected-rating', 'rejected-model']

First example structure:
Keys: ['source', 'prompt', 'chosen', 'chosen-rating', 'chosen-model', 'rejected', 'rejected-rating', 'rejected-model']


## Step 6: Format Dataset for DPO

The DPO trainer needs text format for chosen and rejected responses.
We'll convert the conversational format to plain text.

In [ ]:
def format_chat_to_text(messages):
    """Convert chat format to plain text"""
    if isinstance(messages, list):
        # It's a list of messages
        text_parts = []
        for msg in messages:
            if isinstance(msg, dict) and 'content' in msg:
                text_parts.append(msg['content'])
        return " ".join(text_parts)
    elif isinstance(messages, str):
        # Already text
        return messages
    else:
        return str(messages)

# Format the dataset
def format_dataset(examples):
    prompts = []
    chosen_texts = []
    rejected_texts = []

    for prompt, chosen, rejected in zip(examples['prompt'], examples['chosen'], examples['rejected']):
        # Convert to text format
        prompt_text = format_chat_to_text(prompt)
        chosen_text = format_chat_to_text(chosen)
        rejected_text = format_chat_to_text(rejected)

        prompts.append(prompt_text)
        chosen_texts.append(chosen_text)
        rejected_texts.append(rejected_text)

    return {
        'prompt': prompts,
        'chosen': chosen_texts,
        'rejected': rejected_texts
    }

# Apply formatting
dataset = dataset.map(format_dataset, batched=True, remove_columns=dataset.column_names)

print("✅ Dataset formatted for DPO!")
print(f"\n📊 Dataset Statistics:")
print(f"Total pairs: {len(dataset)}")
print(f"\nFirst example:")
print(f"Prompt: {dataset[0]['prompt'][:100]}...")
print(f"Chosen: {dataset[0]['chosen'][:100]}...")
print(f"Rejected: {dataset[0]['rejected'][:100]}...")

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

✅ Dataset formatted for DPO!

📊 Dataset Statistics:
Total pairs: 500

First example:
Prompt: Can you write a C++ program that prompts the user to enter the name of a country and checks if it bo...
Chosen: Can you write a C++ program that prompts the user to enter the name of a country and checks if it bo...
Rejected: Can you write a C++ program that prompts the user to enter the name of a country and checks if it bo...


## Step 7: Configure DPO Training

### Key DPO Parameters:

1. **beta** (default 0.1): Temperature parameter
   - Higher β = stronger preference signal
   - Lower β = more conservative, closer to SFT model
   - We use 0.1 (standard)

2. **learning_rate**: Much lower than SFT
   - SFT: 2e-4
   - DPO: 5e-5 (recommended)
   - Prevents over-optimization

3. **max_prompt_length** & **max_length**:
   - Prompt: First part (question)
   - Length: Full sequence (prompt + response)

### Important: Use DPOConfig, not TrainingArguments!

In [ ]:
from trl import DPOTrainer, DPOConfig

# Configure DPO training using DPOConfig
dpo_config = DPOConfig(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    warmup_steps = 5,
    max_steps = 60,
    learning_rate = 5e-5,  # Lower than SFT!
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 1,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = "outputs",
    report_to = "none",
    beta = 0.1,  # DPO temperature parameter
    max_length = 1024,  # Max total sequence length
    max_prompt_length = 512,  # Max prompt length
)

# Create DPO trainer
dpo_trainer = DPOTrainer(
    model = model,
    ref_model = None,  # Unsloth handles reference model internally
    args = dpo_config,
    train_dataset = dataset,
    tokenizer = tokenizer,
)

print("✅ DPO Trainer configured!")
print(f"\n📋 Configuration:")
print(f"   Beta (temperature): 0.1")
print(f"   Learning rate: 5e-5")
print(f"   Training pairs: {len(dataset)}")
print(f"   Max steps: 60")

Extracting prompt in train dataset (num_proc=12):   0%|          | 0/500 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=12):   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=12):   0%|          | 0/500 [00:00<?, ? examples/s]

✅ DPO Trainer configured!

📋 Configuration:
   Beta (temperature): 0.1
   Learning rate: 5e-5
   Training pairs: 500
   Max steps: 60


## Step 8: Train with DPO

### What to Monitor During Training:
1. **rewards/chosen**: Reward for preferred responses (should increase)
2. **rewards/rejected**: Reward for rejected responses (should decrease)
3. **rewards/margins**: Difference between chosen and rejected (should increase)
4. **rewards/accuracies**: How often model prefers chosen over rejected (should increase)
5. **loss**: Overall DPO loss (should decrease)

In [ ]:
# Show GPU stats before training
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"Memory used before training: {start_gpu_memory} GB.\n")

print("🚀 Starting DPO training...\n")
print("Watch for these metrics:")
print("  - rewards/chosen: Should increase (model learns to prefer good responses)")
print("  - rewards/rejected: Should decrease (model learns to avoid bad responses)")
print("  - rewards/margins: Should increase (clearer preference)")
print("  - rewards/accuracies: Should increase (correct preference prediction)\n")
print("="*70)

# Train with DPO!
trainer_stats = dpo_trainer.train()

# Show final stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_training = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)

print(f"\n{'='*70}")
print(f"✅ DPO Training completed!")
print(f"Peak memory used: {used_memory} GB ({used_percentage}% of {max_memory} GB)")
print(f"Memory used for training: {used_memory_for_training} GB")
print(f"Training time: {trainer_stats.metrics['train_runtime']:.2f} seconds")
print(f"Final loss: {trainer_stats.metrics['train_loss']:.4f}")
print(f"{'='*70}")

The model is already on multiple devices. Skipping the move to device specified in `args`.


GPU = Tesla T4. Max memory = 14.741 GB.
Memory used before training: 0.193 GB.

🚀 Starting DPO training...

Watch for these metrics:
  - rewards/chosen: Should increase (model learns to prefer good responses)
  - rewards/rejected: Should decrease (model learns to avoid bad responses)
  - rewards/margins: Should increase (clearer preference)
  - rewards/accuracies: Should increase (correct preference prediction)



==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 4,884,480 of 139,400,064 (3.50% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss
1,0.693100,0.000000,0.000000,0.000000,0.000000,-830.013794,-321.914673,5.679506,5.693461,0,0,0
2,0.693100,0.000000,0.000000,0.000000,0.000000,-912.749573,-625.815186,5.237267,5.380050,No Log,No Log,No Log
3,0.693100,0.000000,0.000000,0.000000,0.000000,-475.942383,-390.326965,5.202846,4.661985,No Log,No Log,No Log
4,0.693400,-0.000147,0.000439,0.500000,-0.000586,-633.042053,-640.542725,5.780379,6.440721,No Log,No Log,No Log
5,0.695500,-0.002633,0.001970,0.750000,-0.004604,-809.801025,-726.425049,6.606822,6.626338,No Log,No Log,No Log
6,0.690900,0.005828,0.001331,0.750000,0.004497,-732.598999,-514.890991,6.236343,5.302424,No Log,No Log,No Log
7,0.692200,0.007017,0.005065,0.375000,0.001951,-827.822754,-553.823914,4.053895,4.647731,No Log,No Log,No Log
8,0.692600,0.009614,0.008397,0.250000,0.001217,-629.245239,-492.080475,4.884870,5.163538,No Log,No Log,No Log
9,0.689400,0.010145,0.002609,0.750000,0.007536,-506.873108,-337.382477,5.580442,5.479986,No Log,No Log,No Log
10,0.684500,0.032358,0.014787,0.625000,0.017572,-634.628357,-534.131470,5.884311,5.523564,No Log,No Log,No Log



✅ DPO Training completed!
Peak memory used: 4.541 GB (30.805% of 14.741 GB)
Memory used for training: 4.348 GB
Training time: 161.26 seconds
Final loss: 0.6634


## Step 9: Test DPO Model

Let's test the preference-aligned model!

In [ ]:
# Enable fast inference
FastLanguageModel.for_inference(model)

# Test prompts
test_prompts = [
    "How can I learn programming effectively?",
    "What's the best way to stay healthy?",
    "Explain artificial intelligence to a beginner."
]

print("Testing DPO-aligned model:\n")
print("="*70)

for prompt in test_prompts:
    # Tokenize
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        use_cache=True
    )

    # Decode
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    print(f"\n❓ Prompt: {prompt}")
    print(f"🤖 DPO Response: {response[len(prompt):].strip()}")
    print("="*70)

print("\n💡 Notice how responses should be:")
print("   - More helpful and detailed")
print("   - Better structured")
print("   - More aligned with human preferences")
print("   - Less likely to give low-quality answers")

Testing DPO-aligned model:


❓ Prompt: How can I learn programming effectively?
🤖 DPO Response: 

❓ Prompt: What's the best way to stay healthy?
🤖 DPO Response: I'm a 30-year-old and I've been getting sick a lot lately. I'm a bit of a surfer, but I've been doing this for a while now. I'm not sure what to expect, but I've been getting sick a lot lately. I'm in the hospital now. I have to take a few antibiotics to clear my system. I'm in the hospital for a while longer. I'm not sure what to expect, but I've been getting sick a lot lately. I'm in the hospital now. I have to take a few antibiotics to clear my system. I'm

❓ Prompt: Explain artificial intelligence to a beginner.
🤖 DPO Response: That's a topic that I've been writing about, and I'm not sure if you're interested in that. I'm not sure what you're interested in. Let me know if you're interested in it. I'm sure that there are many other topics in this area that I don't know about. But that's all I can say. Let me know if you're i

## Step 10: Save DPO Model

In [ ]:
# Save DPO-aligned model
model.save_pretrained("smollm2_dpo_aligned")
tokenizer.save_pretrained("smollm2_dpo_aligned")
print("✅ DPO model saved to: smollm2_dpo_aligned/\n")

# Save merged model
model.save_pretrained_merged("smollm2_dpo_merged", tokenizer, save_method="merged_16bit")
print("✅ Merged DPO model saved to: smollm2_dpo_merged/\n")

# Optional: Export to GGUF
model.save_pretrained_gguf("smollm2_dpo_gguf", tokenizer, quantization_method="q4_k_m")
print("✅ GGUF model saved to: smollm2_dpo_gguf/")

✅ DPO model saved to: smollm2_dpo_aligned/

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...


Unsloth: Copying 1 files from cache to `smollm2_dpo_merged`: 100%|██████████| 1/1 [00:00<00:00,  7.37it/s]


Successfully copied all 1 files from cache to `smollm2_dpo_merged`
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:01<00:00,  1.48s/it]


Unsloth: Merge process complete. Saved to `/content/smollm2_dpo_merged`
✅ Merged DPO model saved to: smollm2_dpo_merged/

Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...


Unsloth: Copying 1 files from cache to `smollm2_dpo_gguf`: 100%|██████████| 1/1 [00:00<00:00,  8.23it/s]


Successfully copied all 1 files from cache to `smollm2_dpo_gguf`
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:01<00:00,  1.27s/it]


Unsloth: Merge process complete. Saved to `/content/smollm2_dpo_gguf`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
Unsloth: Successfully installed llama.cpp!
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['SmolLM2-135M-Instruct.F16.gguf']
U

## Summary & Key Takeaways

### What We Accomplished:
✅ Loaded a pre-trained SFT model (SmolLM2-135M-Instruct)
✅ Prepared preference dataset (500 chosen/rejected pairs)
✅ Trained with DPO to align with human preferences
✅ Tested the preference-aligned model
✅ Saved the model in multiple formats

### Key Concepts:

**DPO vs RLHF**:
- DPO is simpler (no reward model needed)
- More stable training
- Faster to implement
- Similar results to RLHF

**When to Use DPO**:
✅ Aligning model with human preferences
✅ Improving response quality
✅ Reducing harmful/unhelpful outputs
✅ After initial SFT training

**DPO Parameters**:
- **beta**: 0.1 (standard), controls preference strength
- **learning_rate**: 5e-5 (much lower than SFT)
- **max_steps**: 30-100 for small datasets

### For Your Video:
1. Explain preference learning concept (chosen vs rejected)
2. Show dataset format with examples
3. Explain why DPO is simpler than RLHF
4. Show training progress
5. Demonstrate improved responses
6. Discuss when to use DPO vs SFT

### Resources:
- DPO Paper: https://arxiv.org/abs/2305.18290
- Ultrafeedback Dataset: https://huggingface.co/datasets/argilla/ultrafeedback-binarized-preferences-cleaned
- Unsloth DPO Docs: https://docs.unsloth.ai/basics/reasoning-grpo-and-rl/reinforcement-learning-dpo-orpo-and-kto